##Required packages installation 

In [13]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


##Librairies import

In [0]:
import pandas as pd
import numpy as np

from hyperopt import hp, fmin, tpe, STATUS_OK

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold
import eli5
from eli5.sklearn import PermutationImportance

##Google Drive connection

In [15]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix/matrix_two/dw_matrix_car


##Data read

In [16]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [17]:
df = df[df['price_currency'] == 'PLN']
df.shape

(106290, 155)

##Feature engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if SUFFIX_CAT in feat: continue
  if isinstance(df[feat][0], list): continue
  df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [19]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  model = model
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

In [0]:
feats = ['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'param_kod-silnika__cat',
'param_model-pojazdu__cat',
'feature_wspomaganie-kierownicy__cat',
'param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_regulowane-zawieszenie__cat',
'feature_system-start-stop__cat',
'feature_światła-led__cat']

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if x is None else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if x is None else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if x is None else int(x.split('cm')[0].replace(' ', '')))

##Model - XGBRegressor

In [24]:
model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[07:41:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:41:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:41:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9449.513980284812, 81.47168211987172)

##Hyperopt

In [35]:
def obj_func(params):
  print('Training with params:')
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

#space
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed': 0
}

#run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params:
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params:
{'colsample_bytree': 0.5, 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params:
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.25, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params:
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params:
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params:
{'colsample_bytree': 

{'colsample_bytree': 0.9,
 'learning_rate': 1,
 'max_depth': 8,
 'subsample': 0.8500000000000001}

##Available XGBRegressor params

In [0]:
xgb_all_params = {
    'base_score': ,
    'booster': ,
    'colsample_bylevel': ,
    'colsample_bynode': ,
    'colsample_bytree': ,
    'gamma': ,
    'importance_type': ,
    'learning_rate': ,
    'max_delta_step': ,
    'max_depth': ,
    'min_child_weight': ,
    'missing': ,
    'n_estimators': ,
    'n_jobs': ,
    'nthread': ,
    'objective: ,
    'random_state': ,
    'reg_alpha': ,
    'reg_lambda': ,
    'scale_pos_weight': ,
    'seed': ,
    'silent': ,
    'subsample': ,
    'verbosity': 
}